# 🗣️ [**AudioToText**](https://github.com/Carleslc/AudioToText)

[![Donate](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/carleslc)

### 🛠 [Whisper by OpenAI](https://github.com/openai/whisper)


## [Step 1] ⚙️ Install the required libraries

Click ▶️ button below to install the dependencies for this notebook.

In [ ]:
!nvidia-smi
!apt-get install libcudnn8
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')
#@title { display-mode: "form" }
import subprocess
from sys import platform as sys_platform

# Install ffmpeg if not present
status, ffmpeg_version = subprocess.getstatusoutput("ffmpeg -version")
if status != 0:
    if sys_platform == 'linux':
        !apt-get -qq update && apt-get -qq install -y ffmpeg
    else:
        print("Install ffmpeg: https://ffmpeg.org/download.html")
else:
    print(ffmpeg_version.split('\n')[0])

# Install faster-whisper and other dependencies
print("Installing Python dependencies...")
!pip install --upgrade pip
!pip install faster-whisper deepgram-sdk pydub typing-extensions
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install openai-whisper
!pip install srt requests tqdm googletrans==4.0.0rc1 httpx aiometer
# https://stackoverflow.com/a/77671445
!apt install libcublas11

## [Step 2] 📁 Upload your audio files to the Files folder

⬅️ Files folder in Google Colab is on the left menu

Almost any audio or video file format is [supported](https://gist.github.com/Carleslc/1d6b922c8bf4a7e9627a6970d178b3a6).

In [ ]:
#@title { display-mode: "form" }
import os
import torch
import math
from faster_whisper import WhisperModel
import subprocess
from pydub import AudioSegment
from pydub.silence import split_on_silence

# select task
task = "Transcribe" #@param ["Transcribe", "Translate to English"]
task = "transcribe" if task == "Transcribe" else "translate"

# set model
use_model = "large-v2" #@param ["tiny", "base", "small", "medium", "large-v1", "large-v2"]

# other parameters
prompt = "" #@param {type:"string"}
coherence_preference = "More coherence, but may repeat text" #@param ["More coherence, but may repeat text", "Less repetitions, but may have less coherence"]
api_key = '' #@param {type:"string"}

# Set device
if api_key:
    print("Using API")
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
else:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {'GPU' if DEVICE == 'cuda' else 'CPU ⚠️'}")
    if DEVICE == "cuda":
        !nvidia-smi -L
    else:
        print("Not using GPU can result in a very slow execution")
        print("Ensure Hardware accelerator by GPU is enabled in Google Colab: Runtime > Change runtime type")

# Load model
if not api_key:
    model_size = use_model
    print(f"\nLoading {model_size} model...")
    model = WhisperModel(model_size, device=DEVICE, compute_type="float16" if DEVICE == "cuda" else "int8")
    print(f"Model {model_size} loaded successfully.\n")

# Set options for faster-whisper
options = {
    'task': task,
    'beam_size': 5,
    'vad_filter': True,
    'initial_prompt': prompt or None,
    'word_timestamps': False,
    'language': None # Auto-detect language
}

In [ ]:
#@title { display-mode: "form" }
import os
import zipfile
import glob
from whisper.utils import get_writer

# Define the root directory to search for video files
directory_to_scan = '.'  # Scans the current directory. Change this if needed.

# List of video file extensions to look for
video_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.webm', '.mka']

# set output folder
output_dir = "audio_transcription"
os.makedirs(output_dir, exist_ok=True)

# --- Find video files, check for existing subtitles, and sort them ---
audio_files = []
all_files = glob.glob(os.path.join(directory_to_scan, '*'))
all_files.sort()  # Sort the files alphabetically

for file_path in all_files:
    if os.path.isfile(file_path):
        name, ext = os.path.splitext(os.path.basename(file_path))
        if ext.lower() in video_extensions:
            # Correct path for checking existing SRT file
            srt_output_path = os.path.join(output_dir, f"{name}.srt")
            if os.path.exists(srt_output_path):
                print(f"Skipping {os.path.basename(file_path)}: Subtitle file already exists.")
            else:
                audio_files.append(file_path)

if not audio_files:
    print("No new video files found to process.")
else:
    # Perform transcription
    if task == "translate":
        print("-- TRANSLATE TO ENGLISH --")
    else:
        print("-- TRANSCRIPTION --")

    # Helper function to correctly save a single SRT file
    def save_srt_file(result, output_dir, output_file_name):
        writer = get_writer("srt", output_dir)
        # Note: whisper's get_writer automatically adds the extension.
        writer(result, output_file_name)
        output_file_path = os.path.join(output_dir, f"{output_file_name}.srt")
        print(f"Saved: {output_file_path}")

    # Process and save each file individually
    for audio_path in audio_files:
        print(f"\nProcessing: {os.path.basename(audio_path)}\n")

        if not api_key:
            # Faster-Whisper
            segments, info = model.transcribe(audio_path, **options)

            transcription_text = []
            result_segments = []
            for segment in segments:
                transcription_text.append(segment.text)
                result_segments.append({
                    'start': segment.start,
                    'end': segment.end,
                    'text': segment.text
                })

            result = {
                'text': "\n".join(transcription_text),
                'segments': result_segments,
                'language': info.language
            }
            print(f"Detected language: {result['language'].title()}\n")

            for segment in result_segments:
                print(f"[{segment['start']:.2f} --> {segment['end']:.2f}] {segment['text']}")

        else:
            # API processing logic would go here
            pass

        # --- Saving logic for a single SRT file ---
        try:
            # Corrected line: pass 'name' to the helper function.
            save_srt_file(result, output_dir, os.path.splitext(os.path.basename(audio_path))[0])
        except Exception as e:
            print(f"Could not save {os.path.splitext(os.path.basename(audio_path))[0]}.srt: {e}")

    # --- New logic: Create a zip file of the subtitles ---
    zip_filename = "subtitles.zip"
    zip_path = os.path.join('.', zip_filename)

    with zipfile.ZipFile(zip_path, 'w') as zf:
        print(f"\nCreating {zip_filename}...")
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zf.write(file_path, os.path.relpath(file_path, output_dir))

    print(f"All subtitles have been archived into {zip_filename}.")